In [31]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from scipy.sparse import hstack

import joblib
import json
import pickle 

In [ ]:
vectorizer = joblib.load('tfidf_vectorizer.pkl')

xgb_model = joblib.load('xgb_model.pkl')

In [33]:
def remove_patterns(text):
    text = re.sub(r'http[s]?://\S+','', text)
    text = re.sub(r'\[.*?\]\(.*?\)','', text)
    text = re.sub(r'@\w+','', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

stemmer = PorterStemmer()

In [ ]:
def stem_tokens(tokens):
    return " ".join(stemmer.stem(str(token)) for token in tokens)

def preprocess_new_text(raw_text):
    text_lower = raw_text.lower()
    text_cleaned = remove_patterns(text_lower)
    tokens = word_tokenize(text_cleaned)
    stemmed_str = stem_tokens(tokens)
    num_chars = len(text_cleaned)
    num_sents = len(sent_tokenize(text_cleaned))
    return stemmed_str, num_chars, num_sents

In [ ]:
new_text = """
I am doing fine how are you?
"""

stemmed_str, num_chars, num_sents = preprocess_new_text(new_text)

new_text_tfidf = vectorizer.transform([stemmed_str])  

import numpy as np

new_text_num = np.array([[num_chars, num_sents]])

new_text_combined = hstack([new_text_tfidf, new_text_num])  

In [ ]:
prediction = xgb_model.predict(new_text_combined)

print("Predicted class:", prediction[0])

c:\Users\brand\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:40:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Predicted class: 3


In [ ]:
'''
0 = 'anxiety'
1 = 'bipolar'
2 = 'depression'
3 = 'normal'
4 = 'personality disorder'
5 = 'stress'
6 = 'suicidal'
'''

"\n0 = 'anxiety'\n1 = 'bipolar'\n2 = 'depression'\n3 = 'normal'\n4 = 'personality disorder'\n5 = 'stress'\n6 = 'suicidal'\n"